In [1]:
pip install cvzone opencv-python mediapipe jupyter

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pygame

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cvzone
import cv2
import numpy as np
import math
import random
from cvzone.HandTrackingModule import HandDetector
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pygame.init()

(5, 0)

In [4]:

cap = cv2.VideoCapture(0)
cap.set(3, 1260)
cap.set(4, 1080)
detector = HandDetector(detectionCon=0.8, maxHands=1)
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
spidermanMask = cv2.imread('spiderman.png', cv2.IMREAD_UNCHANGED)  # Ensure it has alpha channel

I0000 00:00:1753117109.426696 45001175 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1753117109.472905 45002583 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753117109.489028 45002585 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
class SnakeGameClass:
    def __init__(self, pathSnake, point_s, gameover_s):
        self.points = []
        self.lengths = []
        self.currentLength = 0
        self.allowedLength = 50
        self.previousHead = 0, 0

        self.foodImgs = [cv2.resize(cv2.imread(path, cv2.IMREAD_UNCHANGED), (40, 40), interpolation=cv2.INTER_AREA) for path in foodPaths]
        self.currentFoodImg = random.choice(self.foodImgs)
        self.hfood, self.wfood, _ = self.currentFoodImg.shape
        self.foodPoint = 0, 0
        self.randomFoodLocation()

        self.imgSnake = cv2.imread(pathSnake, cv2.IMREAD_UNCHANGED)
        self.imgSnake = cv2.resize(self.imgSnake, (60, 60), interpolation=cv2.INTER_AREA)
        self.hSnake, self.wSnake, _ = self.imgSnake.shape

        self.score = 0
        self.gameOver = False
        with open("score.txt", "r") as f:
            self.maxScore = int(f.read())

        self.point_s = point_s
        self.gameover_s = gameover_s

    def randomFoodLocation(self):
        self.foodPoint = random.randint(200,1000), random.randint(100,500)
        self.currentFoodImg = random.choice(self.foodImgs)  
        self.hfood, self.wfood, _ = self.currentFoodImg.shape

    def update(self, imgMain, currentHead):
        if self.gameOver:
            cvzone.putTextRect(imgMain, "Game Over", [580, 200], scale=2, colorT=(255, 255, 255), colorR=(204, 0, 0))
            cvzone.putTextRect(imgMain, f"Score: {self.score}", [600, 240], scale=2, colorT=(255, 255, 255), colorR=(204, 0, 0))
            self.maxScore = max(self.maxScore, self.score)
            with open("score.txt", "w") as f:
                f.write(str(self.maxScore))
            cvzone.putTextRect(imgMain, "Press Space to Start Again!", [450, 300], scale=2, colorT=(0, 0, 0), colorR=(255, 255, 255))
        else:
            px, py = self.previousHead
            cx, cy = currentHead

            self.points.append([cx, cy])
            distance = math.hypot(cx-px, cy-py)
            self.lengths.append(distance)
            self.currentLength += distance
            self.previousHead = cx, cy

            if self.currentLength > self.allowedLength:
                for i,length in enumerate(self.lengths):
                    self.currentLength -= length
                    self.lengths.pop(i)
                    self.points.pop(i)
                    if self.currentLength < self.allowedLength:
                        break

            rx, ry = self.foodPoint
            if rx < cx < rx +self.wfood and ry < cy < ry + self.hfood:
                self.point_s.play()
                self.randomFoodLocation()
                self.allowedLength += 20
                self.score += 1

            cvzone.putTextRect(imgMain, f"Score: {self.score}", [1200, 20], scale=1, thickness=2, colorT=(255, 255, 255), colorR=(0, 102, 204))

            if self.points:
                for i,point in enumerate(self.points):
                    if i!=0: 
                        color_intensity = int(255 * (i / len(self.points)))
                        cv2.line(imgMain, self.points[i-1], self.points[i],(0, color_intensity, 0), 20)
                hx, hy = self.points[-1]
                imgMain = cvzone.overlayPNG(imgMain, self.imgSnake, (hx - self.wSnake // 2, hy - self.hSnake // 2))

            rx, ry = self.foodPoint
            imgMain = cvzone.overlayPNG(imgMain,self.currentFoodImg, (rx , ry ))

            pts = np.array(self.points[:-2], np.int32)
            pts = pts.reshape((-1, 1, 2))
            mindist = cv2.pointPolygonTest(pts, (cx,cy), True)
        
            if -0.25 <= mindist <= 0.25:
                self.gameover_s.play()
                self.gameOver = True
                self.points = []
                self.lengths = []
                self.currentLength = 0
                self.allowedLength = 50
                self.previousHead = 0, 0
                self.randomFoodLocation()  

        cvzone.putTextRect(imgMain, f"Highest Score: {self.maxScore}", [0, 20], scale=1, thickness=2, colorT=(255, 255, 255), colorR=(51, 153, 102))

        return imgMain

foodPaths = ["banana.png", "apple.png", "food.png"]
point_s = pygame.mixer.Sound("point.wav")
gameover_s = pygame.mixer.Sound("gameOver.wav")
game = SnakeGameClass("snake.png", point_s, gameover_s)

while True:
    success, img = cap.read()
    if not success:
        print("Failed to grab frame from camera")
        break
    img = cv2.flip(img,1)
    
    # Detect faces and overlay Spiderman mask
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    
    for (x, y, w, h) in faces:
        # Adjust mask size preserving aspect ratio
        maskHeight, maskWidth = spidermanMask.shape[:2]
        aspect_ratio = maskWidth / maskHeight

        new_mask_height = int(h * 1.2)  # Increase mask height to cover forehead
        new_mask_width = int(new_mask_height * aspect_ratio)

        mask_resized = cv2.resize(spidermanMask, (new_mask_width, new_mask_height), interpolation=cv2.INTER_AREA)

        # Calculate new position to center mask on face with upward offset
        x1 = x - int((new_mask_width - w) / 2)
        y1 = y - int(h * 0.2)  # Upward offset for forehead
        x2 = x1 + new_mask_width
        y2 = y1 + new_mask_height

        # Ensure coordinates are within frame boundaries
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(img.shape[1], x2)
        y2 = min(img.shape[0], y2)

        # Recalculate resized mask in case cropping is needed
        mask_resized = cv2.resize(spidermanMask, (x2 - x1, y2 - y1), interpolation=cv2.INTER_AREA)

        # Extract alpha channel and blend
        mask_alpha = mask_resized[:, :, 3] / 255.0
        img_alpha = 1.0 - mask_alpha

        for c in range(0, 3):
            img[y1:y2, x1:x2, c] = (mask_alpha * mask_resized[:, :, c] +
                                    img_alpha * img[y1:y2, x1:x2, c])

    cvzone.putTextRect(img, "Use your INDEX finger to move the Snake", [500, 20], scale=1, thickness=2,
                       colorT=(255, 255, 255), colorR=(0, 0, 0))

    hands, img = detector.findHands(img, flipType=False)
    
    if hands:
        lmList = hands[0]['lmList']
        pointIndex = lmList[8][0:2]
        img = game.update(img, pointIndex)
    cv2.imshow("Image", img)
    key = cv2.waitKey(1)

    if key == ord(' '):
        game.gameOver = False
        game.score = 0
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


2025-07-21 22:28:32.999 Python[91274:45001175] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
/Users/ankitkumar/Library/Python/3.9/lib/python/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
